# Sistema de Validação de Identidade com AWS

Este notebook demonstra o funcionamento do sistema de validação de identidade que utiliza:
- **AWS Rekognition** para comparação facial
- **AWS Textract** para extração de texto de documentos
- **S3** para armazenamento temporário de arquivos

O sistema realiza três etapas principais:
1. Comparação facial entre selfie e documento
2. Extração de dados do documento
3. Validação do nome no comprovante de residência


## 1. Importação das bibliotecas necessárias

In [ ]:
import streamlit as st
import boto3
import uuid
import time
from IPython.display import display, HTML, Markdown

In [ ]:
# Configuração do ambiente AWS
# Nota: Em um notebook, você pode usar as credenciais diretamente ou via variáveis de ambiente
# !pip install boto3 streamlit pillow

## 2. Configuração dos clientes AWS

In [ ]:
def configurar_clientes_aws():
    """Configura e retorna os clientes AWS necessários para o aplicativo"""
    rekognition = boto3.client("rekognition", region_name="us-east-1")
    textract = boto3.client("textract", region_name="us-east-1")
    s3 = boto3.client('s3')
    return rekognition, textract, s3

## 3. Função para comparação facial

In [ ]:
def comparar_rostos(rekognition, selfie_bytes, doc_bytes):
    """
    Compara os rostos entre a selfie e o documento

    Args:
        rekognition: Cliente AWS Rekognition
        selfie_bytes: Bytes da imagem da selfie
        doc_bytes: Bytes da imagem do documento

    Returns:
        dict: Resultado da comparação com similaridade
    """
    result = rekognition.compare_faces(
        SourceImage={'Bytes': selfie_bytes},
        TargetImage={'Bytes': doc_bytes},
        SimilarityThreshold=80
    )
    return result

## 4. Função para extrair texto do documento

In [ ]:
def extrair_texto_documento(textract, doc_bytes):
    """
    Extrai texto do documento usando AWS Textract

    Args:
        textract: Cliente AWS Textract
        doc_bytes: Bytes da imagem do documento

    Returns:
        list: Linhas de texto extraídas do documento
    """
    response = textract.detect_document_text(Document={'Bytes': doc_bytes})
    lines = [block['Text'] for block in response['Blocks'] if block['BlockType'] == 'LINE']
    return lines

## 5. Função para processar comprovante de residência

In [ ]:
def processar_comprovante(textract, s3, comprovante_bytes, bucket_name="projeto-validacao"):
    """
    Processa o comprovante de residência usando AWS Textract

    Args:
        textract: Cliente AWS Textract
        s3: Cliente AWS S3
        comprovante_bytes: Bytes do arquivo PDF do comprovante
        bucket_name: Nome do bucket S3

    Returns:
        list: Linhas de texto extraídas do comprovante
    """
    # Código para upload e processamento do PDF
    # ...implementação similar ao app.py...

    # Versão simplificada para demonstração
    file_key = f'temp/{uuid.uuid4()}.pdf'
    s3.put_object(Bucket=bucket_name, Key=file_key, Body=comprovante_bytes)

    response_start = textract.start_document_text_detection(
        DocumentLocation={'S3Object': {'Bucket': bucket_name, 'Name': file_key}}
    )
    job_id = response_start['JobId']

    # Aguardar processamento
    status = 'IN_PROGRESS'
    while status == 'IN_PROGRESS':
        response_get = textract.get_document_text_detection(JobId=job_id)
        status = response_get['JobStatus']
        if status == 'IN_PROGRESS':
            time.sleep(1)

    # Limpar arquivo temporário
    s3.delete_object(Bucket=bucket_name, Key=file_key)

    # Processar resultados
    lines_comprovante = []
    if status == 'SUCCEEDED':
        for item in response_get['Blocks']:
            if item['BlockType'] == 'LINE':
                lines_comprovante.append(item['Text'])

    return lines_comprovante